## Part 1: Obtain Entries from SQL Database

In [1]:
import pymysql
import pandas as pd 

In [2]:
conn = pymysql.connect(host='50.116.46.175',
                      user = 'diig_user',
                      password = 'vL6z3dVRxJbXshBj54og',
                      db = 'theproducebox',
                      cursorclass = pymysql.cursors.DictCursor) 

In [4]:
# get all users from the menu list
menulist_users = pd.read_sql_query("SELECT * from user where user_status_id='5' or user_status_id='9' or user_status_id='11'", conn)

In [5]:
# retain variables of interest
interest = ['user_id', 'fullname', 'user_email', 'signup_date', 'menu_box_type_id', 'ltv_total_order_count', 'ltv_total_sales', 'is_customer']
menulist_users = menulist_users[interest]

In [6]:
# obtain last date of purchase
last_order = []
for user in menulist_users['user_id']:
    sql_call = f"SELECT max(date) FROM `order` where user_id={user}"
    df = pd.read_sql_query(sql_call, conn)
    if(len(df)==0):
        last_order.append(None)
    else:
        last_order.append(df['max(date)'][0]);

In [7]:
# add column for last order date
menulist_users["last_order"] = last_order

## Part 2: RFM Analysis

In [8]:
import numpy as np
from datetime import datetime, timedelta, date

In [9]:
menulist_users['signup_date'] = pd.to_datetime(menulist_users['signup_date'])
menulist_users['last_order'] = pd.to_datetime(menulist_users['last_order'])

In [10]:
# separate new users 
new_user_indexes = []
index = 0
for date in menulist_users['signup_date']:
    if (datetime.now() - date).days <= 31:
        new_user_indexes.append(index)
    index+=1

In [11]:
df = menulist_users
df_new_users = df.iloc[new_user_indexes,:]
df = df.drop(new_user_indexes)

# df_new_users contains all the people that have signed up in the past 30 days
# df contains everyone else

In [12]:
#create a generic user dataframe to keep CustomerID and new segmentation scores
seg_user = pd.DataFrame(df['user_id'].unique())
seg_user.columns = ['user_id']

In [13]:
# recency
user_last_order = df[["user_id", "last_order"]]
user_last_order.columns = ["user_id", "last_order"]

recency = []
for row in user_last_order['last_order']:
    if(pd.isnull(row)):
        recency.append(5000) # arbitrarily assign a high number of days to those that have never purchased
    else:
        recency.append((datetime.now()- row).days) 

user_last_order["recency"] = recency

#merge this dataframe to our new user dataframe
seg_user = pd.merge(seg_user, user_last_order[['user_id','recency']], on='user_id')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [14]:
#build 5 clusters for recency and add it to dataframe

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5)
kmeans.fit(seg_user[['recency']])
seg_user['recency_cluster'] = kmeans.predict(seg_user[['recency']])

#function for ordering cluster numbers
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

seg_user = order_cluster('recency_cluster', 'recency',seg_user,False)

In [15]:
# frequency
#get order counts for each user and create a dataframe with it
user_frequency = df[["user_id","ltv_total_order_count"]]
user_frequency.columns = ['user_id','frequency']

seg_user = pd.merge(seg_user, user_frequency, on='user_id')

In [16]:
#k-means
kmeans = KMeans(n_clusters=5)
kmeans.fit(seg_user[['frequency']])
seg_user['frequency_cluster'] = kmeans.predict(seg_user[['frequency']])

#order the frequency cluster
seg_user = order_cluster('frequency_cluster', 'frequency',seg_user,True)

#see details of each cluster
seg_user.groupby('frequency_cluster')['frequency'].describe()

,count,mean,std,min,25%,50%,75%,max
frequency_cluster,,,,,,,,
0,20.0,34.800000,19.797927,3.0,19.75,36.0,53.25,62.0
1,23.0,96.130435,15.091552,72.0,83.00,99.0,108.50,120.0
2,13.0,152.615385,16.635997,138.0,139.00,146.0,168.00,180.0
3,28.0,216.928571,16.442782,187.0,203.00,217.5,231.00,242.0
4,16.0,276.687500,16.507448,249.0,262.25,280.5,291.00,298.0


In [17]:
#calculate revenue for each customer
user_revenue = df[["user_id", "ltv_total_sales"]]
user_revenue.columns = ['user_id','revenue']

#merge it with our main dataframe
seg_user = pd.merge(seg_user, user_revenue, on='user_id')

In [18]:
#apply clustering
kmeans = KMeans(n_clusters=5)
kmeans.fit(seg_user[['revenue']])
seg_user['revenue_cluster'] = kmeans.predict(seg_user[['revenue']])


#order the cluster numbers
seg_user = order_cluster('revenue_cluster', 'revenue',seg_user,True)

#show details of the dataframe
seg_user.groupby('revenue_cluster')['revenue'].describe()

,count,mean,std,min,25%,50%,75%,max
revenue_cluster,,,,,,,,
0,43.0,1942.837907,984.355212,78.00,1223.010,2130.20,2800.6950,3701.75
1,22.0,5717.864545,1029.053325,4182.26,4872.935,5696.10,6653.1875,7162.69
2,23.0,8823.047391,1041.845902,7452.92,8146.440,8531.05,9366.4150,11052.05
3,9.0,14168.763333,1892.484311,11817.71,12925.480,13382.86,15863.9700,17137.58
4,3.0,24212.730000,1702.240987,22687.01,23294.700,23902.39,24975.5900,26048.79


In [19]:
seg_user['average_fm'] = (seg_user['frequency_cluster'] + seg_user['revenue_cluster']) / 2.0

In [20]:
# segments stored in list seg
new_user_list = df_new_users['user_id'].tolist()
seg = []
for index, row in menulist_users.iterrows():
    if row['user_id'] in new_user_list:
        seg.append("New Customer")
    else:
        seg_row = seg_user.loc[seg_user['user_id'] == row['user_id']]
        if ((seg_row['recency_cluster'] == 4) & (seg_row['average_fm'] >= 3.5)).bool():
            seg.append('Champion')
        elif ((seg_row['recency_cluster'] == 4) & (seg_row['average_fm'] >= 3)).bool():
            seg.append('Loyal Customer')
        elif ((seg_row['recency_cluster'] == 4) & (seg_row['average_fm'] >= 2)).bool():
            seg.append('Potential Loyalist')
        elif ((seg_row['recency_cluster'] == 4) & (seg_row['average_fm'] < 2)).bool():
            seg.append('Promising')
        elif ((seg_row['recency_cluster'] == 3) & (seg_row['average_fm'] >= 2)).bool():
            seg.append('Customers Needing Attention')
        elif (((seg_row['recency_cluster'] >= 0) & (seg_row['recency_cluster'] <= 3)) & (seg_row['average_fm'] >= 3)).bool():
            seg.append('Once High Value')     
        else:
            seg.append('Lost')

### Final DataFrame
After running the next code chunk, the dataframe user_segments will contain one column with user ID and one column with the segment the user is in.

In [22]:
# join user id and segments into new dataframe
user_segments = pd.DataFrame(list(zip(menulist_users['user_id'].tolist(), menulist_users['user_email'].tolist(), seg)),
              columns=['user_id','user_email','segment'])
user_segments

,user_id,user_email,segment
0,6,amber@artsforlifenc.org,Potential Loyalist
1,10,camille@theproducebox.com,Promising
2,12,cindy.liasophia@gmail.com,Customers Needing Attention
3,13,debi.collins@theproducebox.com,Champion
4,17,jenna.phillips@theproducebox.com,Potential Loyalist
...,...,...,...
95,12776,akersmith@gmail.com,Promising
96,12782,akirkpva@gmail.com,Potential Loyalist
97,12784,akj0522@yahoo.com,Promising
98,12785,akmathew98@yahoo.com,Promising


### Selecting a particular subset of customers
You can change the value in the quotes to include any segment that you wish, such as   
- 'Champion'  
- 'Loyal Customer'  
- 'Potential Loyalist'  
- 'Promising'  
- 'Customers Needing Attention'  
- 'Once High Value'  
- 'Lost' 

In [32]:
subset = user_segments.loc[user_segments['segment'] == 'Champion']

If you only want a list of the email addresses, then run the following code chunk.

In [35]:
subset = subset[['user_email']]
subset

,user_email
3,debi.collins@theproducebox.com
8,harperkristin5@gmail.com
9,laurel@theproducebox.com
10,marybeth.cotterell@theproducebox.com
11,ksobel@bellsouth.net
23,kelly.ballard@theproducebox.com
41,pamela.hughes@theproducebox.com
56,a.whitehouse.c@gmail.com
74,adriawilson@yahoo.com


### Saving to a csv
The following code chunk will save the above dataframe. In quotes is the filename that it will save under. 

In [34]:
subset.to_csv('{FILENAME}.csv', index = False)